In [1]:
# -*- coding: utf8 -*-
import codecs
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import itertools as it
import pickle

In [2]:
import datetime
def dateheure():
    return datetime.datetime.utcnow().strftime('%y%m%d%H%M')

typeEchantillon : 
- fixe pour utiliser tailleEchantillon
- variable pour utiliser nombre,increment
- total pour utiliser tout le lexique

In [3]:
filePrefix="MGC-160104"
lexiquePrefix="MGC-160104"
nbEchantillons=6
tailleEchantillon=15000

##Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [4]:
debug=False

In [5]:
bdlexiqueIn = unicode(u"èò")
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO")
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [6]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    return result

In [ ]:
with open("2015-Data/"+lexiquePrefix+'-Verbes2.pkl', 'rb') as input:
    verbes = pickle.load(input)

verbes["phono"]=verbes["phono"].apply(lambda x: recoder(x))

verbes["tirage"]=0
tirages={}
minEchantillon=1

In [7]:
with open("2015-Data/"+lexiquePrefix+'-Tirages.pkl', 'rb') as input:
    verbes = pickle.load(input)
minEchantillon=5

In [8]:
def tirerFormes(limite):
    taille=len(verbes[verbes["tirage"]>0])
    while taille < limite:
        tirage=np.random.choice(verbes.index,p=verbes["prob"])
        verbes.loc[tirage,"tirage"]+=1
        taille=len(verbes[verbes["tirage"]>0])
        if taille%500==0:
            if not taille in tirages:
                tirages[taille]=verbes[verbes["tirage"]>0]["tirage"].sum()
                print "nb tirages %d/%d"%(len(verbes[verbes["tirage"]>0]),verbes[verbes["tirage"]>0]["tirage"].sum()),
                print "%4.2f"%(float(verbes[verbes["tirage"]>0]["tirage"].sum())/len(verbes[verbes["tirage"]>0]))

In [ ]:
%%time
for echantillon in range(minEchantillon,nbEchantillons+1):
    tirerFormes(echantillon*tailleEchantillon)
    extrait=verbes[verbes["tirage"]>0]
    echantillonPrefix="-%02d"%echantillon
    paradigmes=pd.pivot_table(extrait, values='phono', index=['lexeme'], columns=['case'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()
    paradigmes.to_csv(path_or_buf="2015-Data/"+filePrefix+echantillonPrefix+"-paradigmesSuite.csv",encoding="utf8",sep=";")
    extrait.to_csv(path_or_buf="2015-Data/"+filePrefix+echantillonPrefix+"-tirageSuite.csv",encoding="utf8",sep=";")

In [ ]:
verbes.sort_values(by="tirage",ascending=False)

In [10]:
tirages={}

In [ ]:
#with open("2015-Data/"+filePrefix+'-Tirages.pkl', 'wb') as output:
#   pickle.dump(verbes, output, pickle.HIGHEST_PROTOCOL)